Goal is to process cloud data from solcast and create a dataset for ASHD with cloud data from this station


In [1]:
import pandas as pd

In [16]:
solcast_data = pd.read_csv('../../../../data/1. raw/Cloud Data from Solcast/Sydney Airport 30 June 2010 to 1 July 2013/csv_-33.950033_151.181695_fixed_23_0_PT30M.csv')

In [17]:
solcast_data.drop(columns=['period'], inplace=True)

In [18]:
solcast_data.rename(columns={
    'period_end': 'datetime'}, inplace=True
)

In [19]:
solcast_data.index = pd.to_datetime(solcast_data['datetime'])

In [20]:
solcast_data.index

DatetimeIndex(['2010-06-30 00:30:00+10:00', '2010-06-30 01:00:00+10:00',
               '2010-06-30 01:30:00+10:00', '2010-06-30 02:00:00+10:00',
               '2010-06-30 02:30:00+10:00', '2010-06-30 03:00:00+10:00',
               '2010-06-30 03:30:00+10:00', '2010-06-30 04:00:00+10:00',
               '2010-06-30 04:30:00+10:00', '2010-06-30 05:00:00+10:00',
               ...
               '2013-06-30 19:30:00+10:00', '2013-06-30 20:00:00+10:00',
               '2013-06-30 20:30:00+10:00', '2013-06-30 21:00:00+10:00',
               '2013-06-30 21:30:00+10:00', '2013-06-30 22:00:00+10:00',
               '2013-06-30 22:30:00+10:00', '2013-06-30 23:00:00+10:00',
               '2013-06-30 23:30:00+10:00', '2013-07-01 00:00:00+10:00'],
              dtype='datetime64[ns, UTC+10:00]', name='datetime', length=52656, freq=None)

In [21]:
#remove the timezone information from the index
solcast_data.index = solcast_data.index.tz_localize(None)

In [22]:
solcast_data.drop(columns=['datetime'], inplace=True)


In [27]:
solcast_data.cloud_opacity.value_counts().sum()

np.int64(52656)

In [32]:
solcast_data
#from Solcast website, 0 means no cloud, 100 means completely cloudy.

,cloud_opacity
datetime,
2010-06-30 00:30:00,0.0
2010-06-30 01:00:00,0.0
2010-06-30 01:30:00,0.0
2010-06-30 02:00:00,0.0
2010-06-30 02:30:00,0.0
...,...
2013-06-30 22:00:00,0.0
2013-06-30 22:30:00,0.0
2013-06-30 23:00:00,0.0


there is no missing data here on ceilometer sky clear flag. We'll use this instead. 1 means clear sky, 0 means cloudy.

# CREATE ds12

In [29]:
ds4 = pd.read_csv('../../../data/ds4_ashd_with_weather.csv', parse_dates=['datetime'], index_col='datetime')

In [30]:
ds4

,netload_kW,air_temperature_in_degrees_c,relative_humidity_in_percentage,wind_speed_in_km_h
datetime,,,,
2010-07-01 00:00:00,215.792,8.4,66.0,13.0
2010-07-01 00:30:00,193.348,7.9,67.0,13.0
2010-07-01 01:00:00,166.112,7.5,68.0,16.6
2010-07-01 01:30:00,153.506,7.4,69.0,18.4
2010-07-01 02:00:00,140.508,7.2,68.0,20.5
...,...,...,...,...
2013-06-30 21:30:00,321.514,12.4,88.0,14.8
2013-06-30 22:00:00,309.346,12.3,85.0,16.6
2013-06-30 22:30:00,290.918,12.7,84.0,11.2


In [35]:
ds13 = pd.merge(ds4, solcast_data, left_index=True, right_index=True, how='left')

In [36]:
ds13

,netload_kW,air_temperature_in_degrees_c,relative_humidity_in_percentage,wind_speed_in_km_h,cloud_opacity
datetime,,,,,
2010-07-01 00:00:00,215.792,8.4,66.0,13.0,0.0
2010-07-01 00:30:00,193.348,7.9,67.0,13.0,0.0
2010-07-01 01:00:00,166.112,7.5,68.0,16.6,0.0
2010-07-01 01:30:00,153.506,7.4,69.0,18.4,0.0
2010-07-01 02:00:00,140.508,7.2,68.0,20.5,0.0
...,...,...,...,...,...
2013-06-30 21:30:00,321.514,12.4,88.0,14.8,0.0
2013-06-30 22:00:00,309.346,12.3,85.0,16.6,0.0
2013-06-30 22:30:00,290.918,12.7,84.0,11.2,0.0


In [37]:
#check  missing values
ds13['cloud_opacity'].value_counts().sum()

np.int64(52608)

In [38]:
ds13.to_csv('../../../data/ds13_ashd_with_cloud_solcast.csv', index=True)